In [1]:
import os,sys
os.chdir('../')

In [2]:
pwd

'e:\\gemstone'

In [3]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/arpanchakraborty23/GemStone-Price-Prediction.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="arpanchakraborty23"
os.environ["MLFLOW_TRACKING_PASSWORD"]="6824692c47a35333f9eac5b10041d5c8edbcef0"

In [4]:
from src.constant.ymal_path import *
from src.logging.logger import logging
from src.exception.exception import CustomException
from src.utils.utils import create_dir,read_yaml,load_obj

In [12]:
from pathlib import Path
from dataclasses import dataclass

@dataclass
class ModelEvalConfig:
    dir: Path
    test_arr:Path
    model: Path
  

In [13]:
class ConfigManager:
    def __init__(self,
                 congig_file_path=Config_ymal_file_path,
                 schema_file_path=Schema_ymal_file_path) -> None:
        self.config=read_yaml(congig_file_path)
     

        create_dir([self.config.artifacts_root])

    def get_model_eval_config(self):
        config=self.config.model_eval
        

        model_eval_config=ModelEvalConfig(
            dir=config.dir,
            test_arr=config.test_arr,
            model=config.model,
           
        )
        return model_eval_config

In [14]:
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [15]:
import mlflow
import mlflow.sklearn
import numpy as np
import pandas as pd
from urllib.parse import urlparse
from src.utils.utils import save_json


In [16]:
from dotenv import load_dotenv
load_dotenv()
mlflow_uri=os.getenv('MLFLOW_TRACKING_URI')
from mlflow import MlflowException

In [17]:
class ModelEval:
    def __init__(self,config:ModelEvalConfig) -> None:
        self.config=config

    def eval_metrics(self,y_actual,y_pred):
        try:
            acuracy=r2_score(y_actual,y_pred)*100
            MSE=mean_squared_error(y_actual,y_pred)*100

            MAE=mean_absolute_error(y_actual,y_pred)*100
            RMSE=np.sqrt(MSE)

            return MSE,MAE,RMSE,acuracy
        except Exception as e:
            logging.info('error occured',str(e))
            raise CustomException(sys,e)

    def initiating_model_eval(self):
        try:
            logging.info('model evL has started')

            test_array=np.load(self.config.test_arr)
            model=load_obj(self.config.model)
          

           

            x_test=test_array[:,:-1]
            y_test=test_array[:,-1]


            mlflow.set_registry_uri('https://dagshub.com/arpanchakraborty23/GemStone-Price-Prediction.mlflow')

            track_uri_type_store=urlparse(mlflow.get_tracking_uri()).scheme

            print(track_uri_type_store)

            with mlflow.start_run():
                predict=model.predict(x_test)
                MSE,MAE,RMSE,acuracy=self.eval_metrics(y_actual=y_test,y_pred=predict)

                scores={'Rmse':RMSE,'Mse':MSE,'Mae':MAE,'acuracy':acuracy}

                # save_json(file_path=self.config.metrics,data=scores)

                mlflow.log_metrics('Rmse',RMSE)
                mlflow.log_metrics('Mse',MSE)
                mlflow.log_metrics('Mae',MAE)
                mlflow.log_metrics('acuracy',acuracy)


                if track_uri_type_store !='file':
                    mlflow.sklearn.log_model(model,'model',registered_model_name='regressor_model')

                else:
                    mlflow.sklearn.log_model(model,'model')   
                    logging.info('completed model eval')

        except MlflowException as e:
            tb = e.__traceback__
            logging.info(f' Error occured {str(e)}')
            raise CustomException(sys,e) 





            

In [18]:
try:
    config_mager=ConfigManager()
    model_eval_config=config_mager.get_model_eval_config()
    model_eval=ModelEval(config=model_eval_config)
    model_eval.initiating_model_eval()

except Exception as e:
            logging.info(f' Error occured {str(e)}')
            raise CustomException(sys,e) 

https


AttributeError: 'AttributeError' object has no attribute 'exc_info'